In [13]:
import pandas as pd
import os
from tqdm import tqdm

In [16]:
DATA_DIR = "../data/"
TRAIN_IMG = DATA_DIR + "train/"
VAL_IMG = DATA_DIR + "validation/"
TEST_IMG = DATA_DIR + "test/"

In [17]:
names = pd.read_csv(DATA_DIR + 'class-descriptions-boxable.csv', names=['id','label'], usecols=[0,1])
names.head()

,id,label
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [18]:
keep = ['Man', 'Woman', 'Boy', 'Girl', 'House', 'Food', 'Cat', 'Car', 'Human face', 'Airplane', 'Clothing', 'Tree', 'Mobile phone', 'Bottle', 'Insect', 'Horse', 'Train', 'Guitar', 'Drink', 'Book']
choosen_id = list(names[[i in keep for i in names['label']]]['id'])

In [19]:
train = pd.read_csv(DATA_DIR + "oidv6-train-annotations-bbox.csv")
train.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,...,IsDepiction,IsInside,XClick1X,XClick2X,XClick3X,XClick4X,XClick1Y,XClick2Y,XClick3Y,XClick4Y
0,000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,...,0,0,0.148438,0.012500,0.059375,0.195312,0.148438,0.357812,0.587500,0.325000
1,000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,...,0,0,0.025000,0.248438,0.276563,0.214062,0.914062,0.714063,0.782813,0.948438
2,000002b66c9c498e,xclick,/m/01g317,1,0.151562,0.310937,0.198437,0.590625,1,0,...,0,0,0.243750,0.151562,0.310937,0.262500,0.198437,0.434375,0.507812,0.590625
3,000002b66c9c498e,xclick,/m/01g317,1,0.256250,0.429688,0.651563,0.925000,1,0,...,0,0,0.315625,0.429688,0.256250,0.423438,0.651563,0.921875,0.826562,0.925000
4,000002b66c9c498e,xclick,/m/01g317,1,0.257812,0.346875,0.235938,0.385938,1,0,...,0,0,0.317188,0.257812,0.346875,0.307812,0.235938,0.289062,0.348438,0.385938


In [20]:
train_present = set(os.listdir(TRAIN_IMG))
print(len(train_present))

100000


In [21]:
present = [im_id + '.jpg' in train_present for im_id in tqdm(train['ImageID'])]

100%|██████████| 14610229/14610229 [00:07<00:00, 1995823.15it/s]


In [22]:
train_clean = train[present]

In [23]:
print(len(train_clean['ImageID'].unique()))

100000


We only need class id and Boxes coordinates for training so let's clean more

In [24]:
print(train_clean.columns)

Index(['ImageID', 'Source', 'LabelName', 'Confidence', 'XMin', 'XMax', 'YMin',
       'YMax', 'IsOccluded', 'IsTruncated', 'IsGroupOf', 'IsDepiction',
       'IsInside', 'XClick1X', 'XClick2X', 'XClick3X', 'XClick4X', 'XClick1Y',
       'XClick2Y', 'XClick3Y', 'XClick4Y'],
      dtype='object')


In [25]:
need = ['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']
only_needed_train = train_clean[need]

In [28]:
need = [i in choosen_id for i in only_needed_train['LabelName']]
final_clean = only_needed_train[need]

In [30]:
print(len(final_clean['ImageID'].unique())) # good we don't have unusefull image so pipeline seems good

100000


In [31]:
final_clean.to_csv('cleaned_train_bbox.csv')